# Exemplo 1 - Modelando uma mochila em DocPlex

Seja o seguinte modelo:

$max ~~z = \sum_i (c_i \cdot x_i)$

s.t.

$\sum_i (w_i \cdot x_i) \leq C$

Onde:

* $x_i$ indica se o ítem $i$ é colocado na mochila ou não
* $c_i$ é o ganho monetário por incluir $i$ na mochila
* $w_i$ é o peso do ítem $i$
* $C$ é a capacidade da mochila

In [15]:
# Dados de um toy model

ci = [5, 10, 1, 20, 10, 10]
wi = [1, 1, 1, 10, 5, 5]

C = 20

In [1]:
from cplex.callbacks import LazyConstraintCallback
from docplex.mp.callbacks.cb_mixin import *
from docplex.mp.model import Model

In [7]:
# Criação do modelo

model = Model(name="Mochila", log_output=True)

# Definição de variáveis

# z
z = model.z = model.continuous_var(name="z")

# x
xi = model.xi = model.binary_var_list(len(wi), name="x")


Para a **criação de variáveis**, o Docplex disponibiliza um conjunto de métodos. 

Links úteis:
* https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
* https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html

In [16]:
# Definição de restrições

model.maximize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(ci[i] * xi[i] for i in range(len(ci))))

# \sum_i (w_i \cdot x_i) \leq C
model.add_constraint (model.sum(wi[i] * xi[i] for i in range(len(ci))) <= C)

docplex.mp.LinearConstraint[](x_0+x_1+x_2+10x_3+5x_4+5x_5,LE,20)

**Resolvendo o modelo e obtendo os resultados**

In [17]:
solucao = model.solve()

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
1 of 2 MIP starts provided solutions.
MIP start 'm2' defined initial solution with objective 0.0000.
Retaining values of one MIP start for possible repair.
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 2 columns.
MIP Presolve added 1 rows and 1 columns.
MIP Presolve modified 6 coefficients.
Reduced MIP has 2 rows, 6 columns, and 9 nonzeros.
Reduced MIP has 4 binaries, 2 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 3 rows, 7 columns, and 12 nonzeros.
Reduced MIP has 4 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 thre

In [21]:
print("z = ",z.solution_value)
for i in range(len(ci)):
    print(f"x{i}={xi[i].solution_value}")

print("Solution status:")
print(solucao.solve_details.status_code, solucao.solve_details.status)

z =  46.0
x0=1.0
x1=1.0
x2=1.0
x3=1.0
x4=1.0
x5=0
Solution status:
101 integer optimal solution


Material relevante:

*Classe **solution**
https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.solution.html

*Classe **SolveDetails**
https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.sdetails.html#docplex.mp.sdetails.SolveDetails


# Exemplo 2 - Dimensionamento de lotes

$min~~z = \sum_t (h \cdot I_t + s \cdot y_t)$

$I_0 = 0$

$I_t = I_{(t-1)} + x_t - d_t$

$M \cdot y_t \geq x_t ~~~\forall t$

In [36]:
# Dados do toy

di = [0, 100, 0, 200, 50, 100, 0, 100]
h = 1
s = 300

In [43]:
# Criação do modelo

model = Model(name="LotSizing", log_output=True)

# Definição de variáveis

z = model.z = model.continuous_var(name="z")

xt = model.xt = model.continuous_var_list(len(di), name="x")

yt = model.yt = model.binary_var_list(len(di), name="y")

It = model.it = model.continuous_var_list(len(di), name="i")

# Definição de restrições

# Definição de restrições

model.minimize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(
    h * It[t] + s * yt[t]
    for t in range(len(di))
))

# I_t = I_{(t-1)} + x_t - d_t   \forall t > 0
model.add_constraints (
    It[t] == It[t-1] + xt[t] - di[t]
    for t in range(1, len(di))
)

#M \cdot y_t \geq x_t ~~~\forall t
model.add_constraints (
    100000 * yt[t] >= xt[t]
    for t in range(1, len(di))
)

#I0 = 0
It[0].set_ub(0)

0

In [44]:
solucao = model.solve()

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1500.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 3 rows and 6 columns.
MIP Presolve added 2 rows and 0 columns.
Aggregator did 2 substitutions.
Reduced MIP has 12 rows, 17 columns, and 30 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing fixed 0 vars, tightened 4 bounds.
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 6 coefficients.
Reduced MIP has 12 rows, 17 columns, and 30 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation soluti

In [46]:
#Resultados:

print(f"FO: {z.solution_value}")
print("i\tIt\txt\tyt")
for t in range(len(di)):
    print(f"{t}\t{It[t].solution_value}\t{xt[t].solution_value}\t{yt[t].solution_value}")

FO: 1150.0
i	It	xt	yt
0	0	0	0
1	0	100.0	1.0
2	0	0	0
3	150.0	350.0	1.0
4	100.0	0	0
5	0	0	0
6	0	0	0
7	0	100.0	1.0


## Salvando modelo e solução

In [49]:
model.export_as_lp("losizing.lp")

solucao.export_as_sol(".", "lotsizing.sol")

'./lotsizing.sol'

# Implementando callbacks

Vamos supor o modelo anterior:

$min~~z = \sum_t (h \cdot I_t + s \cdot y_t)$

$I_0 = 0$

$I_t = I_{(t-1)} + x_t - d_t$

$M \cdot y_t \geq x_t ~~~\forall t$

Vamos adicionar uma nova restrição:

$x_t \leq 300 ~~~\forall t$

Mas, agora, queremos adicionar essa restrição apenas se ela for violada. Para isso, vamos usar um callback:

(obs: mais exemplos em: https://github.com/IBMDecisionOptimization/docplex-examples/tree/master/examples/mp/callbacks )

In [63]:
di = [0, 100, 0, 200, 50, 100, 0, 100]
h = 1
s = 300

# Criação do modelo

model = Model(name="LotSizing", log_output=True)

# Definição de variáveis

z = model.z = model.continuous_var(name="z")

xt = model.xt = model.continuous_var_list(len(di), name="x")

yt = model.yt = model.binary_var_list(len(di), name="y")

It = model.it = model.continuous_var_list(len(di), name="i")

# Definição de restrições

# Definição de restrições

model.minimize(z)

# z = \sum_i (c_i \cdot x_i)
model.add_constraint (z == model.sum(
    h * It[t] + s * yt[t]
    for t in range(len(di))
))

# I_t = I_{(t-1)} + x_t - d_t   \forall t > 0
model.add_constraints (
    It[t] == It[t-1] + xt[t] - di[t]
    for t in range(1, len(di))
)

#M \cdot y_t \geq x_t ~~~\forall t
model.add_constraints (
    100000 * yt[t] >= xt[t]
    for t in range(1, len(di))
)

#I0 = 0
It[0].set_ub(0)

0

In [72]:
class MeuCallBack(ConstraintCallbackMixin, LazyConstraintCallback):
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)
    
    @print_called('--> lazy constraint callback called: #{0}')
    def __call__(self):

        xt = self.make_solution_from_vars(self.model.xt)
        print("Iniciando loop...")
        for t in range(len(di)):
            print("OK...")
            print(xt.get_value(self.model.xt[t]))
            if xt.get_value(self.model.xt[t]) > 300:
                print("---")
                sol = self.make_complete_solution()
                print("AAAAAA")
                cts = [xt[t] <= 300]
                print("cts=",cts)
                unsats = self.get_cpx_unsatisfied_cts(cts, sol, tolerance=1e-6)
                for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)

                print("Corte adicionado!")
                print(cts)
                break


meu_callback = model.register_callback(MeuCallBack)

solucao = model.solve()



# print(f"FO: {z.solution_value}")
# print("i\tIt\txt\tyt")
# for t in range(len(di)):
#     print(f"{t}\t{It[t].solution_value}\t{xt[t].solution_value}\t{yt[t].solution_value}")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Legacy callback                                  LD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
--> lazy constraint callback called: #1
Iniciando loop...
OK...
x_0
OK...
x_1
OK...
x_2
OK...
x_3
---

Flow cuts applied:  5
Mixed integer rounding cuts applied:  3
Gomory fractional cuts applied:  3

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.05 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branc